In [0]:
# Connexion au drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Analyses niveau des mers 

### Import

In [0]:
# Import csirosealevel.csv > Global mean sea level timeseries from 1993-2019
downloaded = drive.CreateFile({'id': '1wnJKEI1bASumdRn96P3EzfII872RYwix'})
downloaded.GetContentFile('csirosealevel.csv')
dataCSIRO = pd.read_csv('csirosealevel.csv')

In [0]:
# Import sealevel.csv > 1883-present date compared to average of 1993-2008
downloaded = drive.CreateFile({'id': '1FE1IWOr3vKFgJ4lChV0TmSIk6unwZKyY'})
downloaded.GetContentFile('sealevel.csv')
dataSEA = pd.read_csv('sealevel.csv')

### Screening & Cleaning data

#### dataSEA

In [0]:
dataSEA['Year'] = dataSEA['Time'].apply(lambda x: x[:4])
dataSEA = dataSEA.groupby('Year').mean()
dataSEA.head()

,CW_change_mm,UHSLCFD_change_mm
Year,,
1851,NaN,NaN
1852,NaN,NaN
1853,NaN,NaN
1854,NaN,NaN
1855,NaN,NaN


In [0]:
df1 = pd.DataFrame({'change_mm':dataSEA['CW_change_mm'], 'Lab':'CW',  'Year':dataSEA['CW_change_mm'].index})
df2 = pd.DataFrame({'change_mm':dataSEA['UHSLCFD_change_mm'], 'Lab':'UHSLCFD', 'Year':dataSEA['UHSLCFD_change_mm'].index})

dataSEA = pd.concat([df1, df2])

In [0]:
# model 
Xaxis = dataSEA[dataSEA['Lab'] == 'CW'].index
Yaxis = dataSEA['change_mm'][dataSEA['Lab'] == 'CW']
array = np.arange(1880, 2020)

In [0]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

_ = figure(plot_width=900, plot_height=450, title="Augmentation du niveau des mers et océans de 1880 à nos jours (par rapport au niveau moyen de 1993-2008)")
for lab, color in zip(['CW', 'UHSLCFD'], 
                      ['#27496D', '#52AED8']):
  _.line(dataSEA['Year'][dataSEA['Lab'] == lab], dataSEA['change_mm'][dataSEA['Lab'] == lab], line_width=2, color = color, legend_label = lab)
#_.line(x = np.arange(1880, 2020, 1), y = 0, color = '#C3423F', line_width=4, legend_label = 'niveau moyen (1993 à 2008)')
#_.line(x = array, y =  np.poly1d(np.polyfit(x = Xaxis, y = Yaxis, deg = 1))(array), color = '#C0392B', legend_label = 'prédictions', alpha = .5)

_.yaxis.axis_label = 'Elévation (en mm)'
_.legend.title = "Mesures"
_.legend.location = 'top_left'
_.legend.click_policy = "hide"
show(_)